# Imports

In [1]:
import warnings
import os
import sys
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Realod modules
import importlib

In [3]:
import sys
sys.path.append(r'C:\Users\Lenovo\Documents\DSPython\projetscoring')

In [3]:
# Data Ingestion : import des data et autres fonctionnalité
from src.components.data_ingestion import *

In [4]:
# Data Transformation : Transformation des data
from src.components.data_transformation import DataTransformation

In [6]:
from src.components.model_trainer import *

In [ ]:
from src.utilis import save_object, evaluate_models

# Relaod modules

In [30]:
importlib.reload(sys.modules['src.components.data_ingestion'])
from src.components.data_ingestion import *
importlib.reload(sys.modules['src.components.data_transformation'])
from src.components.data_transformation import *
importlib.reload(sys.modules['src.components.model_trainer'])
from src.components.model_trainer import *

# Data base

In [7]:
# 01 - Data Base : 
# Le chemin pour accéder à la base des données se trouve dans les configurations : data_config.py
data_base = DataIngestion()

In [8]:
# Extraction nom de fichiers avec et sans extensions
liste_name, files_liste_name = data_base.get_files_names()

In [9]:
liste_name

['application_test',
 'application_train',
 'bureau',
 'bureau_balance',
 'credit_card_balance',
 'Final_df',
 'HomeCredit_columns_description',
 'installments_payments',
 'POS_CASH_balance',
 'previous_application',
 'sample_submission']

In [21]:
files_liste_name

['application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'credit_card_balance.csv',
 'Final_df.csv',
 'HomeCredit_columns_description.csv',
 'installments_payments.csv',
 'POS_CASH_balance.csv',
 'previous_application.csv',
 'sample_submission.csv']

# Import data

In [22]:
# df = pd.read_csv('C:/Users/Lenovo/Documents/DSPython/data_projet_7/Final_df.csv')

In [10]:
row_data = data_base.import_file(file_name='application_train.csv', reduce_memory_usage = False)

Importation du dataset...
Importation du dataset réussie !


In [11]:
row_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
rapport_df_row_data = RapportDataFrame(row_data, target_column="TARGET", ID_Columns=["SK_ID_CURR"])
# Rapport : row_datarow_data
print("Data Frame : row_data")
print(20 * "--")
rapport_df_row_data.rapport(nan_threshold = 20, return_column_to_keep=False, print_rapport = True)

Data Frame : row_data
----------------------------------------
Nombre de ligne : 307511 --- Nombre de colonnes : 122
----------------------------------------
Le Taux de remplissage total est égal à : 75.6 %
Nombre de colonnes ayant moins de 20% de valeurs manquantes : 72
Le Nombre de features vides est égal à : 0 Features
----------------------------------------
Les Features vides sont : []
Les Features Ayant plus  20%  de valeurs manquantes sont:
['COMMONAREA_MEDI', 'COMMONAREA_AVG', 'COMMONAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_MEDI', 'FONDKAPREMONT_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAPARTMENTS_AVG', 'LIVINGAPARTMENTS_MEDI', 'FLOORSMIN_AVG', 'FLOORSMIN_MODE', 'FLOORSMIN_MEDI', 'YEARS_BUILD_MEDI', 'YEARS_BUILD_MODE', 'YEARS_BUILD_AVG', 'OWN_CAR_AGE', 'LANDAREA_MEDI', 'LANDAREA_MODE', 'LANDAREA_AVG', 'BASEMENTAREA_MEDI', 'BASEMENTAREA_AVG', 'BASEMENTAREA_MODE', 'EXT_SOURCE_1', 'NONLIVINGAREA_MODE', 'NONLIVINGAREA_AVG', 'NONLIVINGAREA

# Data transformation

In [31]:
transformer_row_data= DataTransformation(row_data)

# Train Test Split

In [14]:
X_train, X_test, y_train, y_test = transformer_row_data.initiate_train_test_split(
    target="TARGET",
    train_size=0.30,
    stratification=True)

X_train : done 1/4 ---- Nombre de lignes :215257, Nombre de colonnes:121
y_train : done 2/4 ---- Nombre de lignes :215257
X_test : done 3/4 ---- Nombre de lignes :92254, Nombre de colonnes:121
y_test : done 4/4 ---- Nombre de lignes :92254
####################
Les données sont enregistrées dans le dossier artifacts


In [28]:
X_train.shape

(215257, 121)

# Preprocessing

In [29]:
# Si les data frames sont existants alors exécuter la cellule pour les importer
# X_train,X_test, y_train, y_test = transformer_row_data.get_data_frames()

In [32]:
X_train, X_test,y_train,y_test= transformer_row_data.initiate_data_transformation(
    X_train, 
    X_test, 
    y_train,
    y_test,
    undersampling= True,
    return_train_test_array=False
)

TARGET distribution avant Undersampling
0    197880
1     17377
Name: TARGET, dtype: int64

TARGET distribution après Undersampling
0    17377
1    17377
Name: TARGET, dtype: int64

Les étapes de transformation :
ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaled', StandardScaler())]),
                                 ['CNT_CHILDREN', 'AMT_INCOME_TOTAL',
                                  'AMT_CREDIT', 'AMT_ANNUITY',
                                  'AMT_GOODS_PRICE',
                                  'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
                                  'DAYS_EMPLOYED', 'DAYS_REGISTRATION',
                                  'DAYS_ID_PUBLISH', 'OWN_CAR_AGE',
                                  'FLAG_MOBIL', 'FLAG_EMP_PHONE',
                                  'FLAG_WO...
                                                  StandardScal

In [33]:
X_train.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_missing,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_missing
0,-0.590758,-0.110090,-0.712862,-0.764309,-0.717713,-0.921362,-1.487348,2.334831,-1.013842,-0.983043,...,0.0,0.0,0.0,0.00000,0.0,0.0,2.006254,0.000000,0.0,2.00011
1,-0.590758,-0.088775,0.708242,-0.140152,0.556812,-0.645804,-1.875241,2.334831,-1.836275,-0.745949,...,0.0,0.0,0.0,0.00000,0.0,0.0,2.006254,0.000000,0.0,2.00011
2,-0.590758,0.053319,0.466863,0.551526,0.465775,0.196306,0.768199,-0.441017,-0.274898,-0.068818,...,0.0,0.0,0.0,0.00000,0.0,0.0,2.006254,0.000000,0.0,2.00011
3,0.774439,-0.017728,-1.062905,-1.313698,-0.977820,-0.156569,0.379614,-0.419183,1.134285,1.831877,...,0.0,0.0,0.0,0.00000,0.0,0.0,2.006254,0.000000,0.0,2.00011
4,-0.590758,-0.166927,-1.284530,-1.278126,-1.237927,0.049505,-1.802973,2.334831,-0.961660,-0.711797,...,0.0,0.0,0.0,2.53008,0.0,0.0,0.000000,2.000686,0.0,0.00000


In [34]:
X_test.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_missing,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_missing
0,-0.590758,0.032005,0.594534,0.330518,0.452769,-1.079527,0.139952,-0.420018,1.247924,-0.926561,...,0.0,0.0,0.0,0.0,0.000000,0.0,2.006254,0.000000,0.0,2.00011
1,-0.590758,0.017795,0.150098,0.221167,-0.197499,0.959179,0.054062,-0.427043,1.308512,-0.328242,...,0.0,0.0,0.0,0.0,2.492607,0.0,0.000000,2.000686,0.0,0.00000
2,-0.590758,-0.053252,0.242165,0.404297,0.452769,-1.145863,-1.034115,-0.415618,1.125298,-0.315764,...,0.0,0.0,0.0,0.0,2.492607,0.0,0.000000,2.000686,0.0,0.00000
3,0.774439,-0.088775,-0.944263,-0.461947,-0.847766,-0.824160,1.456246,-0.415603,1.310252,1.006316,...,0.0,0.0,0.0,0.0,2.492607,0.0,0.000000,2.000686,0.0,0.00000
4,-0.590758,-0.152718,-0.881619,-0.097334,-0.847766,-0.642272,-1.857924,2.334831,-0.548559,-0.993551,...,0.0,0.0,0.0,0.0,2.492607,0.0,0.000000,2.000686,0.0,0.00000


In [35]:
y_train

0        0
1        0
2        0
3        0
4        0
        ..
34749    1
34750    1
34751    1
34752    1
34753    1
Name: TARGET, Length: 34754, dtype: int64

In [36]:
y_test.head(-5)

220956    0
51072     0
86405     0
215731    0
190800    0
         ..
111834    0
121159    0
245472    1
125154    0
217239    1
Name: TARGET, Length: 92249, dtype: int64

# Model trainer

In [7]:
# X_train = pd.read_csv('./artifacts/X_train.csv')
# X_test = pd.read_csv('./artifacts/X_test.csv')
# y_train = pd.read_csv('./artifacts/y_train.csv')
# y_test = pd.read_csv('./artifacts/y_test.csv')

In [37]:
# Initiation du fit et trainig 
models = {
                "Logistic Regression" : LogisticRegression(),
                # "Naive Bayes" : GaussianNB(),
                # "Kneighbors Classifier" : KNeighborsClassifier(3),
#                 "model_4" : model_4(),
#                 "model_5" : model_5(),
#                 "model_6" : model_6()
            }
# print(modeltrainer.initiate_model_trainer(train_array=train_arr, test_array=test_arr, models=models))

In [38]:
params = {
        "Logistic Regression": {
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'solver': ['liblinear', 'saga']
        }
    #     "Kneighbors Classifier":{
    #     'n_neighbors': [3, 5, 7],
    #     'weights': ['uniform', 'distance'],
    #     'algorithm': ['ball_tree', 'kd_tree', 'brute']
    # }
}

In [39]:
modeltrainer = ModelTrainer()

**Logistic Regression**

In [40]:
modeltrainer.initiate_model_trainer(X_tr= X_train, y_tr= y_train, X_te= X_test, y_te= y_test, models=models, param=params)

{'Logistic Regression': 0.6852494200793462}